In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.linalg import inv
from scipy.ndimage import gaussian_filter
import os

# ---------------------------------
# Contrast controls (tweak if needed)
# ---------------------------------
SMOOTH_SIGMA = 0.5     # 0 = off; ~0.4–0.8 is gentle
PCTL_LOW = 15          # robust lower percentile for LogNorm
PCTL_HIGH = 99.5       # robust upper percentile for LogNorm
DPI = 180

# -----------------------------
# Amplifier Model (unchanged physics)
# -----------------------------
def self_energy_6node(omega, p, pump=0):
    wS, w1, w2, w3, w4, w5 = p['freqs']
    gamma1, gamma2 = p['gammas']
    J_SB1, J_SB2 = p['J_SB']
    J_B1B2 = p['J_B1B2']
    J_B1B3, J_B1B4, J_B2B4, J_B2B5 = p['J_L1L2']
    J_B3B4, J_B4B5, J_B3B5 = p['J_L2internal']
    g = p['nonlinear_gain']
    J_B3B4_eff = J_B3B4 + g * pump

    M2 = np.array([
        [omega - w3 + 1j*gamma2,   -J_B3B4_eff,          -J_B3B5],
        [-J_B3B4_eff,              omega - w4 + 1j*gamma2, -J_B4B5],
        [-J_B3B5,                  -J_B4B5,              omega - w5 + 1j*gamma2]
    ])
    J_L2 = np.array([
        [J_B1B3, 0],
        [J_B1B4, J_B2B4],
        [0,      J_B2B5]
    ])
    A = J_L2.T @ inv(M2) @ J_L2
    M1 = np.array([
        [omega - w1 + 1j*gamma1 - A[0,0],      -J_B1B2 - A[0,1]],
        [-J_B1B2 - A[1,0],                     omega - w2 + 1j*gamma1 - A[1,1]]
    ])
    J_SB = np.array([J_SB1, J_SB2])
    Sigma = J_SB.T @ inv(M1) @ J_SB
    G_SS = 1 / (omega - wS - Sigma)
    G_L1 = inv(M1) @ J_SB * G_SS
    G_L2 = inv(M2) @ J_L2 @ G_L1
    return G_L2[0]

# -----------------------------
# Sweep Setup
# -----------------------------
params = {
    'freqs': [6.0, 6.49, 6.71, 7.015, 7.2, 7.4],
    'gammas': [0.001, 0.02],
    'J_SB': [0.3, 0.18],
    'J_B1B2': 0.5,
    'J_L1L2': [0.5, 0.42, 0.36, 0.36],
    'J_L2internal': [0.12, 0.09, 0.08],
    'nonlinear_gain': 0.3
}

omega = np.linspace(6.0, 7.5, 300)
sweeps = {
    "J_L1L2": (np.linspace(0.01, 0.6, 100), lambda p,v: [x*v for x in p['J_L1L2']]),
    "gamma1": (np.linspace(0.001, 0.1, 100), lambda p,v: [v, p['gammas'][1]]),
    "gamma2": (np.linspace(-0.1, 0.2, 100), lambda p,v: [p['gammas'][0], v]),
    "J_SB": (np.linspace(0.02, 0.6, 100), lambda p,v: [v, v*0.8]),
    "J_L2internal": (np.linspace(0.02, 0.8, 100), lambda p,v: [x*v for x in p['J_L2internal']]),
    "Pump": (np.linspace(0, 6.0, 100), None)
}

os.makedirs("amplifier_data", exist_ok=True)

# -----------------------------
# Sweep Execution
# -----------------------------
for label, (vals, mod) in sweeps.items():
    grid, ridge = [], []
    for v in vals:
        p = params.copy()
        if label != "Pump":
            key = label if label not in ["gamma1", "gamma2"] else "gammas"
            p[key] = mod(params, v)
            pump = 0.0
        else:
            pump = v
        line = [np.abs(self_energy_6node(w, p, pump))**2 for w in omega]
        grid.append(line)
        ridge.append(omega[np.argmax(line)])

    grid = np.array(grid)
    pd.DataFrame(grid).to_csv(f"amplifier_data/{label}_gain.csv", index=False, header=False)
    pd.DataFrame({"sweep": vals, "ridge": ridge}).to_csv(f"amplifier_data/{label}_ridge.csv", index=False)

# -----------------------------
# High-contrast Composite Plot
# -----------------------------
labels = ["J_L1L2", "gamma1", "gamma2", "J_SB", "Pump", "J_L2internal"]
titles = [
    "Breathing Control: Memory ↔ Release",
    "L1 Coherence Control: Low ↔ Clean Breathing",
    "L2 Sink Efficiency: Optimal ↔ Poor Readout",
    "Qubit-Readout Integration: Stronger ↔ Faster Transfer",
    "Amplification Gain: Pump-Activated Readout",
    "L2 Network Coherence: Internal Amplifier Dynamics"
]
xlabels = ["J_L1L2", "Gamma_1", "Gamma_2", "J_SB", "Pump Power", "J_L2"]
ylabels = ["Frequency (GHz)", "Frequency (GHz)", "Frequency (GHz)", "Frequency (GHz)", "Frequency (GHz)", "Frequency (GHz)"]

def robust_lognorm(arr, low=PCTL_LOW, high=PCTL_HIGH):
    arr = np.clip(arr, 1e-12, None)
    # avoid zero/NaN issues for percentiles
    flat = arr.ravel()
    flat = flat[np.isfinite(flat) & (flat > 0)]
    if flat.size == 0:
        return colors.LogNorm(vmin=1e-12, vmax=1e-10)
    vmin = np.percentile(flat, low)
    vmax = np.percentile(flat, high)
    vmin = max(vmin, 1e-12)
    if not np.isfinite(vmax) or vmax <= vmin:
        vmax = vmin * 100.0
    return colors.LogNorm(vmin=vmin, vmax=vmax)

fig, axes = plt.subplots(2, 3, figsize=(18, 10), constrained_layout=True, dpi=DPI)
for i, label in enumerate(labels):
    gain = pd.read_csv(f"amplifier_data/{label}_gain.csv", header=None).values
    sweep = pd.read_csv(f"amplifier_data/{label}_ridge.csv")["sweep"].values
    ridge = pd.read_csv(f"amplifier_data/{label}_ridge.csv")["ridge"].values

    gain_plot = gaussian_filter(gain, sigma=SMOOTH_SIGMA) if SMOOTH_SIGMA > 0 else gain
    norm = robust_lognorm(gain_plot)

    ax = axes[i//3, i%3]
    im = ax.imshow(
        gain_plot.T,
        aspect='auto',
        origin='lower',
        extent=[sweep[0], sweep[-1], omega[0], omega[-1]],
        norm=norm  # let matplotlib choose default colormap (no explicit colors)
    )
    # Ridge overlay
    ax.plot(sweep, ridge, linewidth=1.2, alpha=0.9)

    ax.set_title(titles[i], fontsize=12)
    ax.set_xlabel(xlabels[i])
    ax.set_ylabel(ylabels[i])

# one shared colorbar
cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.95, pad=0.02)
cbar.set_label("|G_B3S|² (robust log scale)", fontsize=12)

plt.savefig("amplifier_data/composite_grid_highcontrast.png", bbox_inches="tight")
plt.close()

print("Saved: amplifier_data/composite_grid_highcontrast.png")


Saved: amplifier_data/composite_grid_highcontrast.png
